# Прогнозирование биологического ответа

Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).

Данные представлены в формате CSV.  Каждая строка представляет молекулу. 

- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 

- Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

Предварительная обработка не требуется, данные уже закодированы и нормализованы.

В качестве метрики будем использовать F1-score.

Необходимо обучить две модели: логистическую регрессию и случайный лес. Далее нужно сделать подбор гиперпараметров с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (GridSeachCV, RandomizedSearchCV, Hyperopt, Optuna) хотя бы по разу, максимальное количество итераций не должно превышать 50.

In [2]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

#%matplotlib inline
# plt.style.use('seaborn')

## Базовая оптимизация

### Знакомство с данными и их исследование

In [9]:
# Загрузка данных
data = pd.read_csv('data/_train_sem09 (1).csv')
data.head(4)


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0


Создаем матрицу наблюдений $X$ и вектор ответов $y$

In [ ]:
# Создаём новый датафрейм только с нужными столбцами
# X = data.iloc[:, [1, -1]]  # Дескрипторы D1-D1776
X = data[['D1', 'D1776']] # Дескрипторы D1-D1776
y = data['Activity']  # Целевая переменная

display(X.head(4))
display(y.head(4))

,D1,D1776
0,0.000000,0
1,0.366667,0
2,0.033300,0
3,0.000000,0


0    1
1    1
2    1
3    1
Name: Activity, dtype: int64

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение). 

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

#### **Логистическая регрессия**

Сначала посчитаем модель с параметрами по умолчанию и оценим метрику:

In [63]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)

#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

#Делаем предсказание для тренировочной выборки
y_train_predict = log_reg.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_predict = log_reg.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_predict)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_predict)))

f1_score на тренировочным наборе: 0.69
f1_score на тестовом наборе: 0.69


### Логистическая регрессия с GridSearchCV

In [64]:
from sklearn.model_selection import GridSearchCV

param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
]
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 

#Делаем предсказание для тренировочной выборки
y_train_pred = grid_search.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_pred = grid_search.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 625 ms
Wall time: 25.7 s
f1_score на тренировочным наборе: 0.70
f1_score на тестовом наборе: 0.70
Наилучшие значения гиперпараметров: {'C': 0.7, 'penalty': 'l2', 'solver': 'lbfgs'}


c:\Users\alexD\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
70 fits failed out of a total of 140.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
35 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\alexD\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\alexD\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\alexD\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.p

Ознакомиться с итоговой полученной моделью можно с помощью best_estimator_. 

In [26]:
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))

Наилучшая модель:
LogisticRegression(C=0.7, max_iter=1000, random_state=42)


А наилучшее значение точности кросс-валидации (значение точности, усредненное по всем разбиениям для данной комбинации гиперпараметров) – в атрибуте best_score_. 

In [28]:
print("Наилучшее значение точности при кросс-валидаци: {:.2f}".format(grid_search.best_score_))

Наилучшее значение точности при кросс-валидаци: 0.55


*Вывод*

**Метрику удалось улучшить!**

Использование GridSearchCV помогает найти оптимальные гиперпараметры для модели логистической регрессии. Благодаря этому можно значительно улучшить производительность модели.

Это означает, что оптимальная модель логистической регрессии была найдена с:

- Параметром регуляризации C = 0.7 (умеренная регуляризация);
- Регуляризацией типа l2 (Ridge-регуляризация);
- Использованием алгоритма оптимизации lbfgs (Limited-memory BFGS – эффективный алгоритм для задач с большим числом параметров).


#### **Случайный лес**

Проделаем аналогичное для RandomForestClassifier().
Сначала посчитаем модель с параметрами по умолчанию и оценим метрику:

In [65]:
#Создаем объект класса случайный лес
rfc = ensemble.RandomForestClassifier(random_state=42)

#Обучаем модель
rfc.fit(X_train, y_train)

#Делаем предсказание для тренировочной выборки
y_train_predict = rfc.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_predict = rfc.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_predict)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_predict)))

f1_score на тренировочным наборе: 0.65
f1_score на тестовом наборе: 0.64


### Случайный лес с RandomizedSearchCV

In [31]:
from sklearn.model_selection import RandomizedSearchCV

In [66]:
param_distributions = {'n_estimators': list(range(80, 200, 30)),
              'min_samples_leaf': [5],
              'max_depth': list(np.linspace(20, 40, 10, dtype=int))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 

#Делаем предсказание для тренировочной выборки
y_train_predic = random_search_forest.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_predic = random_search_forest.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_predic)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_predic)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

CPU times: total: 703 ms
Wall time: 23.6 s
f1_score на тренировочным наборе: 0.70
f1_score на тестовом наборе: 0.70
Наилучшие значения гиперпараметров: {'n_estimators': 110, 'min_samples_leaf': 5, 'max_depth': 26}


*Вывод*

**Метрику удалось улучшить!**

Использование  RandomizedSearchCV помогает найти оптимальные гиперпараметры для модели Случайный лес. Благодаря этому можно значительно улучшить производительность модели.

Это означает, что оптимальная модель Случайный лес была найдена с:

- n_estimators=  110;
- min_samples_leaf = 5;
- max_depth =  35.

##  Продвинутая оптимизация

### Случайный лес с Hyperopt

In [67]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [68]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

Интерфейс hyperopt отличается от Grid или RandomizedSearch, поэтому нам нужно создать функцию для минимизации, она должна принимать словарь значений гиперпараметров и возвращать значение целевой функции.

In [69]:
# зафксируем random_state
random_state = 42
def hyperopt_rfc(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

In [70]:
%%time
# начинаем подбор гиперпараметров

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rfc, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [00:21<00:00,  1.07s/trial, best loss: -0.7008316475612497]
Наилучшие значения гиперпараметров {'max_depth': 19.0, 'min_samples_leaf': 3.0, 'n_estimators': 132.0}
CPU times: total: 12.5 s
Wall time: 21.4 s


In [71]:
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)

#Делаем предсказание для тренировочной выборки
y_train_predi = model.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_predi = model.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_predi)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_predi)))

f1_score на тренировочным наборе: 0.70
f1_score на тестовом наборе: 0.70


*Вывод*

**Метрику удалось улучшить!**

Hyperopt помогает найти оптимальные гиперпараметры для модели Случайный лес. Благодаря этому можно значительно улучшить производительность модели.

Это означает, что оптимальная модель Случайный лес была найдена с:

- n_estimators=  132;
- min_samples_leaf = 3;
- max_depth =  19.

### Случайный лес с Optuna

In [72]:
import optuna

Настроим оптимизацию гиперпараметров для алгоритма случайного леса.


In [75]:
def optuna_rfс(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [78]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rfс, n_trials=20)

# выводим результаты Наилучшие значения гиперпараметров
print("Наилучшие значения гиперпараметров {}".format(study.best_params))

[I 2024-12-11 14:40:02,702] A new study created in memory with name: RandomForestClassifier
C:\Users\alexD\AppData\Local\Temp\ipykernel_20032\874632992.py:3: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
C:\Users\alexD\AppData\Local\Temp\ipykernel_20032\874632992.py:4: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
C:\Users\alexD\AppData\Local\Temp\ipykernel_20032\874632992.py:5: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
[I 2024-12-11 14:40:04,628] Trial 0 finished with value: 0.6472303206997084 and parameters: {'n_estimators': 152, 'max_depth': 27, 'min_samples_leaf': 5

Наилучшие значения гиперпараметров {'n_estimators': 131, 'max_depth': 26, 'min_samples_leaf': 2}
CPU times: total: 13 s
Wall time: 20.2 s


In [79]:
# рассчитаем точность для тестовой выборки и обучающей выборке
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )

model.fit(X_train, y_train)

#Делаем предсказание для тренировочной выборки
y_train_pr = model.predict(X_train)
#Делаем предсказание для тестовой выборки
y_test_pr = model.predict(X_test)
 
print('f1_score на тренировочным наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pr)))
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pr)))

f1_score на тренировочным наборе: 0.70
f1_score на тестовом наборе: 0.70


*Вывод*

**Метрику удалось улучшить!**

Optuna помогает найти оптимальные гиперпараметры для модели Случайный лес. Благодаря этому можно значительно улучшить производительность модели.

Это означает, что оптимальная модель Случайный лес была найдена с:

- n_estimators=  131;
- min_samples_leaf = 2;
- max_depth =  26.